# Three counterexamples for the ratio of doublings W as a statistic of relative fitness

Consider a competition growth cycle between a wildtype population $N_1$ and a mutant strain $N_2$ from start $t=t_0$ to saturation $t=t_f$.


#### Definition of the ratio of doublings

\begin{align}
 W = \frac{\log N_2(t_f)/N_2(t_0)}{\log N_1(t_f)/N_1(t_0)}
\end{align}

#### Definition of the integrated selection coefficient

\begin{align}
 s = \log N_2(t_f)/N_1(t_f) - \log N_2(t_0)/N_1(t_0)
\end{align}



#### Definition of the normalized selection coefficient

\begin{align}
 S = s / \log \big( N_1(t_f)/N_1(t_0) \big)
\end{align}




### Parameters of this notebook 

In [ ]:
import pandas as pd           

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from scipy import integrate
from scipy import stats
import random


In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/epistasis/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())


### Test settings

In [ ]:
from selection_coefficient import Problem_M3, get_ODE_solution, plot_solution

In [ ]:
### define solver params
solver_params = {'t_final' : 100000, 'timestep' : 10, 'adaptive_timewindow' : True, 'rtol' : 1e-8, 'atol' : 1e-12, 'scoeff_atol':1e-8, 'scoeff_rtol':1e-6}

In [ ]:
strain_params = {'lam':[6.,3.], 'g':[1.,1.5], 'Y':[1.,0.5]}

problem = Problem_M3(**strain_params, x =0.4)
sol = get_ODE_solution(problem, **solver_params)

In [ ]:

fig, ax = plt.subplots(figsize = (9,6))
plot_solution(sol,ax)
ax.legend(loc = 'center right')


In [ ]:
from selection_coefficient import estimate_s21, estimate_s21_timecourse

In [ ]:
def estimate_W21(sol):

    y_end = sol['y'][:,-1]

    y_start = sol['y'][:,0]

    

    N1_0, N2_0,R_0 = y_start

    N1_f, N2_f,R_f = y_end

    

    W = np.log(N2_f/N2_0) / np.log(N1_f/N1_0)

    return W

    

In [ ]:
estimate_W21(sol)

In [ ]:
def estimate_S21_capital(sol):
    y_end = sol['y'][:,-1]
    y_start = sol['y'][:,0]
    
    N1_0, N2_0,R_0 = y_start
    N1_f, N2_f,R_f = y_end
    
    s = np.log(N2_f/N1_f) - np.log(N2_0/N1_0)
    
    WT_foldchange = np.log(N1_f/N1_0)
    
    S_capital = s/WT_foldchange
    
    return S_capital
    

In [ ]:
estimate_S21_capital(sol)

In [ ]:
def estimate_p_timecourse(sol):
    
    N1_0, N2_0 = sol.y[0,0], sol.y[1,0]
    
    N1_t, N2_t = sol.y[0,:], sol.y[1,:]
    
    p_t = np.divide(N2_t, np.add(N1_t,N2_t))
    
    
    return sol.t, p_t




    

In [ ]:
_, p_t = estimate_p_timecourse(sol)

print(p_t)

In [ ]:
### choose linewidth
lw = 3

### choose wildtype color
color_wt = 'grey'

### choose default mutant color
color_mut = 'tab:blue'

In [ ]:
### set boundary for graphics
t_final = 10
ymax = 10
ymin = 0.001



### Example 0: wildtype unter bacteriostatic growth

In [ ]:


### set initial conditions 
initial_conditions = {'x':0.01, 'N_0':0.1, 'R_0':0.3}

### set final tim efor plot
tmax = 12

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = axes[0]


### set parameters for mutant A
strain_params = {'lam':[2.,2.], 'g':[0.,0.7], 'Y':[1.,1.]}
problem = Problem_M3(**strain_params, **initial_conditions)
sol = get_ODE_solution(problem, **solver_params)

## plot wildtype

ax.plot(sol.t, sol.y[0], color = 'tab:grey', lw=lw, label = 'wildtype')
## plot mutant

ax.plot(sol.t, sol.y[1], color = 'tab:orange', lw=lw, label = 'mutant')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_0={p0:.2f}, x_f={pf:.2f}, s={s_21:.3f}, W={W_21:.3f}$'
ax.set_title(title, loc = 'right')



ax = axes[1]
### set parameters for mutant A
strain_params = {'lam':[2.,2.], 'g':[0.,0.7], 'Y':[1.,0.05]}

problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw=lw, label = 'wildtype')
## plot mutant
ax.plot(sol.t, sol.y[1], color = 'tab:purple', lw=lw, label = 'mutant')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_0={p0:.2f}, x_f={pf:.2f}, s={s_21:.3f}, W={W_21:.3f}$'
ax.set_title(title, loc = 'right')



for ax in axes:
    ax.set_xlim(0,tmax)
    ax.set_ylim(0.0005,0.5)
    ax.set_yscale('log')
    
    ax.legend(loc = 'lower right')
    
    ax.set_xlabel('time')
    ax.set_ylabel('population size [OD]')
    
fig.tight_layout()
fig.savefig(FIG_DIR + "example_bacteriostatic.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Example 1: lag times

In [ ]:

strain_params = {'lam':[2.,1.], 'g':[1.,1.], 'Y':[1.,1.]}





In [ ]:
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = axes[0]

### set initial conditions for environment A
initial_conditions = {'x':0.5, 'N_0':0.01, 'R_0':0.5}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype

ax.plot(sol.t, sol.y[0], color = color_wt, lw=lw, label = 'wildtype $N_1$')
## plot mutant

ax.plot(sol.t, sol.y[1], color = color_mut, lw=lw, label = 'mutant $N_2$')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f={pf:.2f}, s={s_21:.3f},W={W_21:.3f}$'
ax.set_title(title, loc = 'right')



ax = axes[1]

### set initial conditions for environment B
initial_conditions['R_0'] = 2
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw=lw, label = 'wildtype $N_1$')
## plot mutant
ax.plot(sol.t, sol.y[1], color = color_mut, lw=lw, label = 'mutant $N_2$')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f={pf:.2f}, s={s_21:.3f},W={W_21:.3f}$'
ax.set_title(title, loc = 'right')



for ax in axes:
    ax.set_xlim(0,t_final)
    ax.set_ylim(ymin,ymax)
    ax.set_yscale('log')
    
    ax.legend(loc = 'lower right')
    
    ax.set_xlabel('time')
    ax.set_ylabel('population size [OD]')
    
fig.tight_layout()
fig.savefig(FIG_DIR + "example_lagtimes_growthcurves.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Example 2: growth rates 

In [ ]:

strain_params = {'lam':[3.,3.], 'g':[1.,1.5], 'Y':[1.,1.]}





In [ ]:
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = axes[0]

### set initial conditions for environment A
initial_conditions = {'x':0.5, 'N_0':0.01, 'R_0':0.5}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype

ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'mutant $N_2$')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f={pf:.2f}, s={s_21:.3f},W={W_21:.3f}$'
ax.set_title(title, loc = 'right')



ax = axes[1]

### set initial conditions for environment B
initial_conditions['R_0'] = 2
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'mutant $N_2$')

### compute selection coefficient
s_21 = estimate_s21(sol)
### estimate log doubling ratio
W_21 = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f={pf:.2f}, s={s_21:.3f},W={W_21:.3f}$'
ax.set_title(title, loc = 'right')


for ax in axes:
    ax.set_xlim(0,t_final)
    ax.set_ylim(ymin,ymax)
    ax.set_yscale('log')
    
    ax.legend(loc = 'lower right')
    
    ax.set_xlabel('time')
    ax.set_ylabel('population size [OD]')
    
fig.tight_layout()

fig.savefig(FIG_DIR + "example_growthrates_growthcurves.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### Example 3: epistasis with lag time

In [ ]:
from matplotlib import gridspec

In [ ]:
### set initial conditions for environment A
initial_conditions = {'x':0.5, 'N_0':0.01, 'R_0':0.5}

## plot
pad = 0.4
gridspec_kw = {"width_ratios":[1,1,1], "wspace":pad, 'hspace':0.0}
fig , axes = plt.subplots(1,3, figsize = (3*0.8*FIGWIDTH_TRIPLET+3*pad, 0.7*FIGHEIGHT_TRIPLET),
                          gridspec_kw=gridspec_kw)

ax = axes[0]
strain_params = {'lam':[2.,1.], 'g':[1.,1], 'Y':[1.,1.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'tab:blue'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'lag-time mutant $N_3$')

### compute selection coefficient
s_21_lag = estimate_s21(sol)
### estimate log doubling ratio
W_21_lag = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_lag = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_lag:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_lag:.3f}'
ax.set_title(title, loc = 'right')



ax = axes[1]

strain_params = {'lam':[2.,2.], 'g':[1.,1], 'Y':[1.,2.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'tab:red'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'cell-yield mutant $N_2$')

### compute selection coefficient
s_21_yield = estimate_s21(sol)
### estimate log doubling ratio
W_21_yield = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_yield = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_yield:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_yield:.3f}'
ax.set_title(title, loc = 'right')


ax = axes[2]

strain_params = {'lam':[2.,1.], 'g':[1.,1], 'Y':[1.,2.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'purple'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'double mutant $N_5$')

### compute selection coefficient
s_21_lag_yield = estimate_s21(sol)
### estimate log doubling ratio
W_21_lag_yield = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_lag_yield = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_lag_yield:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_lag_yield:.3f}'
ax.set_title(title, loc = 'right')



for ax in axes:
    ax.set_xlim(0,t_final)
    ax.set_ylim(ymin,ymax)
    ax.set_yscale('log')
    
    #ax.legend(loc = 'lower right')
    
    ax.set_xlabel('time')
    ax.set_ylabel('absolute abundance [OD]')
    
#fig.tight_layout()

fig.savefig(FIG_DIR + "example_epistasis_lagtime+yield.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:
### plot epistasis diagrams


## plot
pad = 0.5
gridspec_kw = {"width_ratios":[1,1], "wspace":pad, 'hspace':0.0}
fig , axes = plt.subplots(1,2, figsize = (2*0.6*FIGWIDTH_TRIPLET+1*pad, 0.7*FIGHEIGHT_TRIPLET),
                          gridspec_kw=gridspec_kw)

ax = axes[1]

ax.plot([0,1], [0,s_21_lag], color = 'tab:grey',  ls = '--')
ax.plot([1,2], [s_21_yield,s_21_lag_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.plot([0,1], [0,s_21_yield], color = 'tab:grey', ls = '--')
ax.plot([1,2], [s_21_lag,s_21_lag_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.scatter([0], [0], color = color_wt, marker = 'o', zorder = 10, s =80)
ax.scatter([1], [s_21_lag], color = 'tab:blue', marker = 'o', zorder = 10, s =80)
ax.scatter([1], [s_21_yield], color = 'tab:red', marker = 'o', zorder = 10, s =80)
ax.scatter([2], [s_21_lag_yield], color = 'tab:purple', marker = 'o', zorder = 10, s =80)

## fix axis labels
ax.set_xticks([0,1,2])
ax.set_xticklabels(['wild-type', 'single\nmutant', 'double\nmutant'])
ax.set_ylabel('fitness per-cycle: '+ r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')


ax = axes[0]

ax.plot([0,1], [0,S_21_capital_lag], color = 'tab:grey', ls = '--')
ax.plot([1,2], [S_21_capital_yield,S_21_capital_lag_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.plot([0,1], [0,S_21_capital_yield], color = 'tab:grey', ls = '--')
ax.plot([1,2], [S_21_capital_lag,S_21_capital_lag_yield], color = 'tab:grey', zorder = -1, ls = '--')

#ax.scatter([2], [S_21_capital_lag_yield], color = 'tab:purple', marker = 'o')
ax.scatter([0], [0], color = color_wt, marker = 'o', zorder = 10, s =80)
ax.scatter([1], [S_21_capital_lag], color = 'tab:blue', marker = 'o', zorder = 10, s =80)
ax.scatter([1], [S_21_capital_yield], color = 'tab:red', marker = 'o', zorder = 10, s =80)
ax.scatter([2], [S_21_capital_lag_yield], color = 'tab:purple', marker = 'o', zorder = 10, s =80)

## fix axis labels
ax.set_xticks([0,1,2])
ax.set_xticklabels(['wild-type', 'single\nmutant', 'double\nmutant'])
ax.set_ylabel('fitness per-generation: '+ r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')

fig.savefig(FIG_DIR + "example_epistasis_lag+yield.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Example 3: epistasis with growth rate

In [ ]:
from matplotlib import gridspec

In [ ]:
### set initial conditions for environment A
initial_conditions = {'x':0.5, 'N_0':0.01, 'R_0':0.5}

## plot
pad = 0.4
gridspec_kw = {"width_ratios":[1,1,1], "wspace":pad, 'hspace':0.0}
fig , axes = plt.subplots(1,3, figsize = (3*0.8*FIGWIDTH_TRIPLET+3*pad, 0.7*FIGHEIGHT_TRIPLET),
                          gridspec_kw=gridspec_kw)

ax = axes[0]
strain_params = {'lam':[2.,2.], 'g':[1,1.2], 'Y':[1.,1.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'tab:blue'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'lag-time mutant $N_3$')

### compute selection coefficient
s_21_growth = estimate_s21(sol)
### estimate log doubling ratio
W_21_growth = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_growth = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_growth:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_growth:.3f}'
ax.set_title(title, loc = 'right')



ax = axes[1]

strain_params = {'lam':[2.,2.], 'g':[1.,1], 'Y':[1.,4.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'tab:red'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'cell-yield mutant $N_2$')

### compute selection coefficient
s_21_yield = estimate_s21(sol)
### estimate log doubling ratio
W_21_yield = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_yield = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_yield:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_yield:.3f}'
ax.set_title(title, loc = 'right')


ax = axes[2]

strain_params = {'lam':[2.,2.], 'g':[1.,1.2], 'Y':[1.,4.]}
problem = Problem_M3(**strain_params, **initial_conditions)

sol = get_ODE_solution(problem, **solver_params)

## plot wildtype
ax.plot(sol.t, sol.y[0], color = color_wt, lw = lw, label = 'wildtype $N_1$')
## plot mutant
color_mut = 'purple'
ax.plot(sol.t, sol.y[1], color = color_mut, lw = lw, label = 'double mutant $N_5$')

### compute selection coefficient
s_21_growth_yield = estimate_s21(sol)
### estimate log doubling ratio
W_21_growth_yield = estimate_W21(sol)
### estimate normalized selection coefficient
S_21_capital_growth_yield = estimate_S21_capital(sol)
### estimate final frequency
_, p_t = estimate_p_timecourse(sol)
p0,pf = p_t[0],p_t[-1]

## build title
title = f'$x_f$={pf:.2f}, ' +r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$'+f'={s_21_growth_yield:.3f}, ' + \
        r'$s^{\mathrm{logit}}_{\mathrm{gen}}$'+f'={S_21_capital_growth_yield:.3f}'
ax.set_title(title, loc = 'right')



for ax in axes:
    ax.set_xlim(0,t_final)
    ax.set_ylim(ymin,ymax)
    ax.set_yscale('log')
    
    #ax.legend(loc = 'lower right')
    
    ax.set_xlabel('time')
    ax.set_ylabel('absolute abundance [OD]')
    
#fig.tight_layout()

fig.savefig(FIG_DIR + "example_competitions_growthrate+yield.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:

## plot
pad = 0.5
gridspec_kw = {"width_ratios":[1,1], "wspace":pad, 'hspace':0.0}
fig , axes = plt.subplots(1,2, figsize = (2*0.6*FIGWIDTH_TRIPLET+1*pad, 0.7*FIGHEIGHT_TRIPLET),
                          gridspec_kw=gridspec_kw)

ax = axes[1]

ax.plot([0,1], [0,s_21_growth], color = 'tab:grey',  ls = '--')
ax.plot([1,2], [s_21_yield,s_21_growth_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.plot([0,1], [0,s_21_yield], color = 'tab:grey', ls = '--')
ax.plot([1,2], [s_21_growth,s_21_growth_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.scatter([0], [0], color = color_wt, marker = 'o', zorder = 10, s =80)
ax.scatter([1], [s_21_growth], color = 'tab:blue', marker = 'o', zorder = 10, s =80)
ax.scatter([1], [s_21_yield], color = 'tab:red', marker = 'o', zorder = 10, s =80)
ax.scatter([2], [s_21_growth_yield], color = 'tab:purple', marker = 'o', zorder = 10, s =80)

## fix axis labels
ax.set_xticks([0,1,2])
ax.set_xticklabels(['wild-type', 'single\nmutant', 'double\nmutant'])
ax.set_ylabel('fitness per-cycle: '+ r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')


ax = axes[0]

ax.plot([0,1], [0,S_21_capital_growth], color = 'tab:grey', ls = '--')
ax.plot([1,2], [S_21_capital_yield,S_21_capital_growth_yield], color = 'tab:grey', zorder = -1, ls = '--')

ax.plot([0,1], [0,S_21_capital_yield], color = 'tab:grey', ls = '--')
ax.plot([1,2], [S_21_capital_growth,S_21_capital_growth_yield], color = 'tab:grey', zorder = -1, ls = '--')

#ax.scatter([2], [S_21_capital_growth_yield], color = 'tab:purple', marker = 'o')
ax.scatter([0], [0], color = color_wt, marker = 'o', zorder = 10, s =80)
ax.scatter([1], [S_21_capital_growth], color = 'tab:blue', marker = 'o', zorder = 10, s =80)
ax.scatter([1], [S_21_capital_yield], color = 'tab:red', marker = 'o', zorder = 10, s =80)
ax.scatter([2], [S_21_capital_growth_yield], color = 'tab:purple', marker = 'o', zorder = 10, s =80)

## fix axis labels
ax.set_xticks([0,1,2])
ax.set_xticklabels(['wild-type', 'single\nmutant', 'double\nmutant'])
ax.set_ylabel('fitness per-generation: '+ r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')

fig.savefig(FIG_DIR + "example_epistasis_growthrate+yield.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Example 4: long-term trait evolution

In [ ]:
t_vec = np.linspace(0,5e4, num = 200) # time in generations

def eval_growthrate(t,  g0 = 1):
    g0 = 1 # per hour # todo: update with real LTEE vlaue
    g_limit = 1.5 # per hour, limiting value reached
    t_limit = 25000 # time in generations where limiting value is reached
    
    assert t >=0, 'time needs to be positive'
    if t < t_limit:
        g = t/t_limit *(g_limit - g0) + g0
    else:
        g = g_limit
        
    return g

g0 = 1
g_vec = np.array([eval_growthrate(v,g0=g0) for v in t_vec])

def eval_lagtime(t, lag0= 3):
    # lag time in hours # todo: update with real LTEE vlaue
    lag_limit = 0 # per hour, limiting value reached
    t_limit = 15000 # time in generations where limiting value is reached
    
    assert t >=0, 'time needs to be positive'
    if t < t_limit:
        lag = t/t_limit *(lag_limit - lag0) + lag0
    else:
        lag = lag_limit
        
    return lag

lag0  = 3
lag_vec = np.array([eval_lagtime(v, lag0) for v in t_vec])


def eval_yield(t, Y0= 3):
    # lag time in hours # todo: update with real LTEE vlaue
    Y_limit = 0 # per hour, limiting value reached
    t_limit = 100000 # time in generations where limiting value is reached
    
    assert t >=0, 'time needs to be positive'
    if t < t_limit:
        Y = t/t_limit *(Y_limit - Y0) + Y0
    else:
        Y = Y_limit
        
    return Y

Y0  = 3
Y_vec = np.array([eval_yield(v, Y0) for v in t_vec])

In [ ]:
### make a dataframe

df_evolution = pd.DataFrame(data = np.vstack([t_vec, g_vec,lag_vec,Y_vec]).T, columns= ['t','g', 'lag', 'Y'])

In [ ]:
def row2statistics(row):
    strain_params = {'lam':[lag0, row['lag']], 'g':[g0,row['g']], 'Y':[Y0,row['Y']]}
    initial_conditions = {'x':0.5, 'N_0':0.01, 'R_0':0.5}
    problem = Problem_M3(**strain_params, **initial_conditions)
    
    ### need to solve for statistic W
    sol = get_ODE_solution(problem, **solver_params)
    ###
    s21 = estimate_s21(sol)
    W21 = estimate_W21(sol)
    
    return {'s':s21, 'W': W21}

## test
row = df_evolution.iloc[0] ## expect zero selection coefficient
series = pd.Series(row2statistics(row))
print(series)

In [ ]:
%%time
df_evolution['s'] = np.nan
df_evolution['W'] = np.nan

for i in range(df_evolution.shape[0]):
    row = df_evolution.iloc[i]
    statistics = row2statistics(row)
    df_evolution.at[i,'s'] = statistics['s']
    df_evolution.at[i,'W'] = statistics['W']
    

In [ ]:
fig, axes = plt.subplots(2,1, figsize = (2*FIGHEIGHT_TRIPLET, 2*0.7*FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0]
ax.plot(t_vec,g_vec/1, label = 'growth rate', lw = lw)
ax.plot(t_vec,lag_vec/lag0, label = 'lag time', lw = lw)
ax.plot(t_vec,Y_vec/Y0, label = 'yield', lw = lw)

#ax.legend(loc = 'upper left', frameon = False)
ax.set_ylabel('evolved trait\nrelative to ancestor')
ax.set_ylim(-0.05,2.05)

ax = axes[1]
label = 'fitness per-generation: $s^{\mathrm{logit}}_{\mathrm{gen}}$'
ax.plot(t_vec, df_evolution['W']-1, color = 'tab:red', label = label, lw = lw)
label = 'fitness per-cycle: $s^{\mathrm{logit}}_{\mathrm{cycle}}$'
ax.plot(t_vec, df_evolution['s'], color = 'tab:grey', label = label,lw =lw)

ax.set_ylabel('parwise competition\nwith ancestor(50:50)')
ax.set_xlabel('time [#generations in evolution experiment]')
ax.set_xlim(0,t_vec[-1])
ax.legend(frameon=False)

#fig.tight_layout()
fig.savefig(FIG_DIR + f'false_positive_fitness_increase_over_long_times.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:
fig, axes = plt.subplots(2,1, figsize = (1.5*FIGHEIGHT_TRIPLET, 2*0.7*FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0]
ax.plot(t_vec,g_vec/1, label = 'growth rate', lw = lw)
ax.plot(t_vec,lag_vec/lag0, label = 'lag time', lw = lw)
ax.plot(t_vec,Y_vec/Y0, label = 'yield', lw = lw)

#ax.legend(loc = 'upper left', frameon = False)
ax.set_ylabel('evolved trait\nrelative to ancestor')
ax.set_ylim(-0.05,2.05)

ax = axes[1]
label = 'fitness per-generation: $s^{\mathrm{logit}}_{\mathrm{gen}}$'
ax.plot(t_vec, df_evolution['W']-1, color = 'tab:red', label = label, lw = lw)
label = 'fitness per-cycle: $s^{\mathrm{logit}}_{\mathrm{cycle}}$'
ax.plot(t_vec, df_evolution['s'], color = 'tab:grey', label = label,lw =lw)

#ax.set_ylabel('parwise competition\nwith ancestor(50:50)')
ax.set_ylabel('fitness\nrelative to ancestor')
ax.set_xlabel('time [# generations in evolution experiment]')
ax.set_xlim(0,t_vec[-1])
ax.legend(frameon=False)

#fig.tight_layout()
fig.savefig(FIG_DIR + f'false_positive_fitness_increase_over_long_times_short_plot.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)